# Model Development

## Data

In [0]:
def get_sampled_features(labeled_features, sample_size=.25):
    sample = labeled_features.select('customer_id').distinct().sample(sample_size)
    labeled_features.join(sample, 'customer_id')
    
    return labeled_features

In [0]:
labeled_features = spark.read.parquet('gs://h-and-m-tx/features/labels-bq/')

## ALS Model

In [0]:
def als_model(df, n=12):
    indexer = StringIndexer(inputCol="customer_id", outputCol="customer_id_enc") 
    df = indexer.fit(df).transform(df)
    
    # Cast all fields to integers

    df = df.withColumn('customer_id_enc', col('customer_id_enc').cast('int')) \
        .withColumn('article_id', col('article_id').cast('int')) \
        .withColumn('narticles', col('narticles').cast('int'))
    
    (training, test) = df.randomSplit([0.8, 0.2])
    
    als = ALS(maxIter=5, 
          implicitPrefs=True, # Set implicit preferences to True since these are purchases
          userCol='customer_id_enc', 
          itemCol='article_id', 
          ratingCol='narticles', 
          coldStartStrategy='drop')
    model = als.fit(training)
    predictions = model.transform(test)

    reco_at_n = model.recommendForAllUsers(n)
    
    return reco_at_n
    
    


In [0]:
from pyspark.sql.functions import count

# Get only one year data and summarize/count by number of articles purchased per customer
# where(col('t_dat') < '2019-01-01') \

als_features = (
    labeled_features
    .groupBy('customer_id_labeled', 'article_id_labeled')
    .agg(count('*').alias("narticles"))
    #.where(col('narticles') >= 5)
)

print(f'length als_features {als_features.count()}')

length als_features 6802387


In [0]:
# Label encode or transform string customer_id to integers

# from pyspark.ml.feature import StringIndexer

# indexer = StringIndexer(inputCol="customer_id", outputCol="customer_id_enc") 
# tx_all = indexer.fit(tx_all).transform(tx_all)

In [0]:
# Cast all fields to integers

als_features = (
    als_features
    .withColumn('customer_id', col('customer_id').cast('int'))
    .withColumn('article_id', col('article_id').cast('int'))
    .withColumn('narticles', col('narticles').cast('int'))
)
# tx_all.printSchema()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-112983542186388> in <cell line: 4>()
      3 als_features = (
      4     als_features
----> 5     .withColumn('customer_id', col('customer_id').cast('int'))
      6     .withColumn('article_id', col('article_id').cast('int'))
      7     .withColumn('narticles', col('narticles').cast('int'))

NameError: name 'col' is not defined

In [0]:
(training, test) = als_features.randomSplit([0.8, 0.2])

In [0]:
from pyspark.ml.recommendation import ALS


als = ALS(maxIter=5, 
          implicitPrefs=True, # Set implicit preferences to True since these are purchases
          userCol='customer_id_labeled', 
          itemCol='article_id_labeled', 
          ratingCol='narticles', 
          coldStartStrategy='drop')
model = als.fit(training)

In [0]:
predictions = model.transform(test)
predictions.show()

+-------------------+------------------+---------+-------------+
|customer_id_labeled|article_id_labeled|narticles|   prediction|
+-------------------+------------------+---------+-------------+
|                 22|              9665|        1| 3.3949275E-4|
|                 22|             39341|        1| 1.4857114E-4|
|                 26|             61452|        1| 2.4067063E-4|
|                103|             71199|        1|  9.859968E-4|
|                115|             58963|        1|-7.0271045E-7|
|                122|            101736|        1| 1.5463504E-4|
|                185|             57337|        1|  0.010834838|
|                185|             84939|        1| 0.0016879551|
|                185|             89181|        1| 0.0037684764|
|                185|             98405|        1| 0.0053701503|
|                185|             70964|        1| 0.0026249841|
|                185|            101021|        1| 0.0012080718|
|                209|    

### Measure ALS Model

In [0]:
def get_top_n_articles(customer_id_enc, n):
    return tx_all.where(col('customer_id_enc') == customer_id_enc).select('*').limit(n)

In [0]:
users = test.select('customer_id_labeled').distinct()
#reco_at_n = model.recommendForAllUsers(12)
reco_at_n = model.recommendForUserSubset(users, 12)
reco_at_n.createOrReplaceTempView("reco_at_n")
print(f'reco_at_n count: {reco_at_n.count()}')

reco_at_n count: 208548


In [0]:
from pyspark.sql import functions as F

actuals = test.groupBy('customer_id_labeled').agg(F.collect_list('article_id_labeled').alias('actuals'))
print(f'actuals count: {actuals.count()}')

actuals count: 244642


In [0]:
predicted = spark.sql("""
select customer_id_labeled, recommendations.article_id_labeled as predicted
from reco_at_n
""")

print(f'number of predicted: {predicted.count()}')

number of predicted: 208548


In [0]:
actuals.printSchema()
predicted.printSchema()

root
 |-- customer_id_labeled: long (nullable = true)
 |-- actuals: array (nullable = false)
 |    |-- element: long (containsNull = false)

root
 |-- customer_id_labeled: integer (nullable = false)
 |-- predicted: array (nullable = true)
 |    |-- element: integer (containsNull = true)



In [0]:
predicted_actuals = actuals.join(predicted, actuals.customer_id_labeled ==  predicted.customer_id_labeled, "inner")
#print(f'predicted_actuals count: {predicted_actuals.count()}')

predicted_list = predicted_actuals \
    .select('predicted').rdd.flatMap(lambda x: x).collect()

actuals_list = predicted_actuals \
    .select('actuals').rdd.flatMap(lambda x: x).collect()

In [0]:
display(predicted_actuals)

customer_id_labeled,actuals,customer_id_labeled,predicted
22,"List(9665, 39341)",22,"List(70125, 3088, 42515, 46335, 81670, 81668, 1712, 70124, 81669, 58428, 3707, 42584)"
26,List(61452),26,"List(1713, 14220, 14233, 14226, 2250, 1712, 12567, 12580, 53834, 14221, 9763, 12584)"
103,List(71199),103,"List(81669, 42584, 81668, 70125, 3088, 58428, 70124, 79683, 81670, 87277, 24808, 3085)"
115,List(58963),115,"List(1712, 24809, 3088, 42515, 57581, 81668, 58428, 81670, 81669, 1713, 55527, 14233)"
122,List(101736),122,"List(70125, 3707, 45744, 17022, 17021, 70124, 2250, 56633, 58428, 61534, 47247, 57337)"
185,"List(57337, 84939, 89181, 98405, 70964, 101021)",185,"List(58428, 17022, 1712, 17021, 81669, 3707, 81668, 1713, 70125, 70124, 81670, 9763)"
209,"List(36365, 62231, 74041, 89183, 14220, 23091, 57288, 58702, 76840, 82418, 83591, 90231, 99073, 99752, 3337, 9090, 17405, 21123, 46950, 47941, 54368, 57796, 14237, 47361, 55307, 61796, 83306)",209,"List(14220, 14233, 43989, 24238, 14226, 18561, 49867, 1713, 66967, 17360, 1712, 24809)"
224,List(20551),224,"List(42515, 75, 813, 42584, 15981, 17405, 67483, 17021, 67436, 37150, 64574, 76)"
291,"List(33351, 69522, 73223, 94537, 28424, 46291, 59284, 78378, 88413, 1486, 4979, 8373, 38265, 42150, 53119, 64767, 87464, 98740, 4076, 4177, 87507)",291,"List(14220, 14233, 18561, 1713, 14226, 43989, 2250, 24808, 70125, 17360, 49867, 75776)"
329,"List(2430, 4942, 8608, 9768, 35820, 45432, 46565, 52442, 53320, 54504, 57594, 58722, 58753, 58842, 61297, 62632, 62634, 77222, 79340, 79908, 89880, 99877, 102871, 103012, 103505, 17033, 41236, 47357, 48558, 59947, 60388, 64639, 64871, 68186, 69056, 69061, 76406, 78874, 80094, 93411, 94747, 95580, 96450, 100850, 2333, 9763, 9783, 15106, 22082, 25099, 32249, 32658, 33402, 55542, 60787, 61980, 62630, 66129, 72856, 77742, 81442, 85950, 91616, 101370, 3566, 15296, 35821, 37076, 42515, 43159, 43703, 43989, 44606, 45272, 45278, 50141, 57601, 58165, 60257, 63402, 66967, 73174, 73491, 75248, 76095, 76229, 76558, 81719, 81757, 83173, 92362, 93278, 94277, 94344, 103819)",329,"List(24808, 24809, 14220, 14233, 43989, 15981, 67436, 17021, 17022, 42584, 18561, 58428)"


In [0]:
actuals_list

Out[15]: [[51450, 49825, 55937, 56633, 66062, 100355, 20113, 76408, 15966, 88479],
 [16886,
  20679,
  25422,
  37011,
  98816,
  47989,
  58553,
  63906,
  10468,
  21903,
  30049,
  37008,
  41416,
  60135,
  60394,
  71150,
  30624,
  58219,
  59088,
  62821,
  77223],
 [62141, 102525, 76237],
 [16093, 39195],
 [51752],
 [51824, 53833],
 [74875, 75784, 94956, 15997, 44488, 70989, 6651, 58146, 71959, 75785],
 [15484],
 [73517],
 [44509],
 [55391,
  38633,
  59344,
  6656,
  17022,
  47363,
  71007,
  96396,
  98954,
  17043,
  20565,
  46574,
  74169,
  74173],
 [65506],
 [70282],
 [63195],
 [96114],
 [91436],
 [50275, 61691],
 [46310, 91525, 32464, 58962, 100171, 1712, 12651, 46383, 46413, 57810, 58523],
 [88105],
 [7226,
  71128,
  2775,
  18487,
  77036,
  91709,
  92530,
  56934,
  57799,
  67331,
  72874,
  87833,
  29718,
  42515],
 [91019, 12567, 12800],
 [17650,
  17368,
  28863,
  89362,
  1486,
  15796,
  42626,
  47247,
  54664,
  59324,
  62894,
  70984],
 [53264,
  67164

In [0]:
from pyspark.mllib.evaluation import RankingMetrics

def calculate_metrics(actuals, predict, show=False):
    labels_list = list(zip(predict, actuals))
    labels = spark.sparkContext.parallelize(labels_list)
    
    metrics = RankingMetrics(labels)
    
    mapat12 = round(metrics.meanAveragePrecisionAt(12),6)
    mapat5 = round(metrics.meanAveragePrecisionAt(5),6)
    
    if show == True:
        print(f'Precision at 12 {mapat12}')
        print(f'Precision at 5 {mapat5}')
    
    return mapat12, mapat5

In [0]:
calculate_metrics(actuals_list, predicted_list, show=True)

/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
Precision at 12 0.002142
Precision at 5 0.002304
Out[17]: (0.002142, 0.002304)

In [0]:
popular = labeled_features.groupBy('article_id_labeled').agg(count("article_id_labeled").alias("narticles")).sort('narticles', ascending=False).limit(12)

#popular.select('article_id_labeled').rdd.flatMap(lambda x: x).collect()

In [0]:
# print(len(actuals_list))
# labels_list = (popular_predict_list, actuals_list)

popular_predict_list = [popular.select('article_id_labeled').rdd.flatMap(lambda x: x).collect()] * len(actuals_list)

calculate_metrics(actuals_list, popular_predict_list, show=True)

# labels_list = list(zip(popular_predict_list, actuals_list))

# labels = spark.sparkContext.parallelize(labels_list)
# metrics = RankingMetrics(labels)
# print(f'Precision at 12 {metrics.meanAveragePrecisionAt(12)}')
# print(f'Precision at 5 {metrics.meanAveragePrecisionAt(5)}')

/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
Precision at 12 0.003401
Precision at 5 0.003721
Out[19]: (0.003401, 0.003721)

### Revenue instead of count of articles

In [0]:
transactions \
    .groupBy('customer_id', 'article_id') \
    .agg(sum('price').alias("revenue")) \
    .show()

In [0]:
transactions_full = transactions.join(articles, transactions.article_id ==  articles.article_id, "inner")
transactions_full.cache()

print(f'number of transactions {transactions_full.count()}')

In [0]:
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline

features = ['product_code', 
          'product_type_no',
          'graphical_appearance_no',
          'colour_group_code',
          'perceived_colour_value_id',
          'perceived_colour_master_id',
          'department_no',
          'garment_group_no']

for feature in features:
    transactions_full = transactions_full.withColumn(feature, col(feature).cast('int'))
    
print(transactions_full.printSchema())

#Convert qualificationIndex and genderIndex
onehot_encoder = OneHotEncoder(inputCols=['product_code', 
                                          'product_type_no',
                                          'graphical_appearance_no',
                                          'colour_group_code',
                                          'perceived_colour_value_id',
                                          'perceived_colour_master_id',
                                          'department_no',
                                          'garment_group_no'], 
                               outputCols=['product_code_vec', 
                                          'product_type_no_vec',
                                          'graphical_appearance_no_vec',
                                          'colour_group_code_vec',
                                          'perceived_colour_value_id_vec',
                                          'perceived_colour_master_id_vec',
                                          'department_no_vec',
                                          'garment_group_no_vec'], )

#Merge multiple columns into a vector column
vector_assembler = VectorAssembler(inputCols=['product_code_vec', 
                                          'product_type_no_vec',
                                          'graphical_appearance_no_vec',
                                          'colour_group_code_vec',
                                          'perceived_colour_value_id_vec',
                                          'perceived_colour_master_id_vec',
                                          'department_no_vec',
                                          'garment_group_no_vec'], 
                                   outputCol='features')
#Create pipeline and pass it to stages
pipeline = Pipeline(stages=[
           onehot_encoder,
           vector_assembler
])
#fit and transform
transactions_feats = pipeline.fit(transactions_full).transform(transactions_full)
transactions_feats.show()

# |-- article_id: string (nullable = true)
#  |-- product_code: string (nullable = true)
#  |-- prod_name: string (nullable = true)
#  |-- product_type_no: string (nullable = true)
#  |-- product_type_name: string (nullable = true)
#  |-- product_group_name: string (nullable = true)
#  |-- graphical_appearance_no: string (nullable = true)
#  |-- graphical_appearance_name: string (nullable = true)
#  |-- colour_group_code: string (nullable = true)
#  |-- colour_group_name: string (nullable = true)
#  |-- perceived_colour_value_id: string (nullable = true)
#  |-- perceived_colour_value_name: string (nullable = true)
#  |-- perceived_colour_master_id: string (nullable = true)
#  |-- perceived_colour_master_name: string (nullable = true)
#  |-- department_no: string (nullable = true)
#  |-- department_name: string (nullable = true)
#  |-- index_code: string (nullable = true)
#  |-- index_name: string (nullable = true)
#  |-- index_group_no: string (nullable = true)
#  |-- index_group_name: string (nullable = true)
#  |-- section_no: string (nullable = true)
#  |-- section_name: string (nullable = true)
#  |-- garment_group_no: string (nullable = true)
#  |-- garment_group_name: string (nullable = true)
#  |-- detail_desc: string (nullable = true)

## Deep Learning

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [0]:
# identify categorical feature columns
# identify label column
label_col = 'label'
cat_features = ('product_group_name','graphical_appearance_name','customer_id','article_id')
cat_features_labeled = [x + "_labeled" for x in cat_features]

In [0]:
labeled_features = spark.read.parquet('gs://h-and-m-tx/features/labels-bq/')
labeled_features = labeled_features.drop(*cat_features)
labeled_features.cache()

In [0]:
# fraction to hold for training
train_fraction = 0.6
 
# sample data, stratifying on labels, for training
train = (
  labeled_features
    .sampleBy(label_col, fractions={0: train_fraction, 1: train_fraction})
  )
 
# split remaining data into validation & testing datasets (with same stratification)
valid = (
  labeled_features
    .join(train, on='id', how='leftanti') # not in()
    .sampleBy(label_col, fractions={0:0.5, 1:0.5})
  )
 
test = (
  labeled_features
    .join(train, on='id', how='leftanti') # not in()
    .join(valid, on='id', how='leftanti') # not in()
  )

#print(f'train: {train.count()} valid: {valid.count()} test: {test.count()}')

In [0]:
train.write.parquet('gs://h-and-m-tx/features/train', mode='overwrite')
valid.write.parquet('gs://h-and-m-tx/features/valid', mode='overwrite')
test.write.parquet('gs://h-and-m-tx/features/test', mode='overwrite')

## Train Model

In [0]:
train = spark.read.parquet('gs://h-and-m-tx/features/train')
valid = spark.read.parquet('gs://h-and-m-tx/features/valid')
test = spark.read.parquet('gs://h-and-m-tx/features/test')

In [0]:
import pyspark.sql.functions as F
# identify categorical feature columns
cat_features = cat_features_labeled
# capture keys for each of the categorical feature columns
cat_keys={}
for col in cat_features:
    cat_keys[col] = (
        labeled_features
        .selectExpr('{0} as key'.format(col))
        .distinct()
        .orderBy('key')
        .groupBy()
          .agg(F.collect_list('key').alias('keys'))
        .collect()[0]['keys']
    )

# all other columns (except id) are continous features
num_features = labeled_features.drop(*(['id',label_col]+cat_features)).columns

In [0]:
!pip install tensorflow
!pip install petastorm
!pip install hyperopt
!pip install mlflow

In [0]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
 
from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec
 
from hyperopt import hp, fmin, tpe, SparkTrials, STATUS_OK, space_eval
 
import mlflow
from mlflow.tracking import MlflowClient
 
import platform
 
import numpy as np
import pandas as pd
 
import datetime
import os
import requests

In [0]:
# configure temp cache for petastorm files
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:///dbfs/tmp/instacart_wide_deep/pstorm_cache') # the file:// prefix is required by petastorm
 
# persist dataframe data to petastorm cache location
train_pstorm = make_spark_converter(train.repartition(4))  
valid_pstorm = make_spark_converter(valid.repartition(4)) 
test_pstorm = make_spark_converter(test.repartition(4)) 

In [0]:
def get_data_specs(epochs=1, batch_size=128):
    epochs = int(epochs)
    batch_size = int(batch_size)
    # define functions to transform data into req'ed format
    def get_input_fn(dataset_context_manager):
    
        # re-structure a row as ({features}, label)
        def _to_tuple(row): 
            features = {}
            for col in cat_features + num_features:
                features[col] = getattr(row, col)
            return features, getattr(row, label_col)
    
        def fn(): # called by estimator to perform row structure conversion
            return dataset_context_manager.__enter__().map(_to_tuple)
      
        return fn
 
    # access petastorm cache as tensorflow dataset
    train_ds = train_pstorm.make_tf_dataset(batch_size=batch_size)
    valid_ds = valid_pstorm.make_tf_dataset()
  
    # define spec to return transformed data for model training & evaluation
    train_spec = tf.estimator.TrainSpec(
        input_fn=get_input_fn(train_ds), 
        max_steps=int( (train_pstorm.dataset_size * epochs) / batch_size )
    )
    eval_spec = tf.estimator.EvalSpec(
        input_fn=get_input_fn(valid_ds)
    )
  
    return train_spec, eval_spec

In [0]:
# Verify Specs

# retrieve specs
specs = get_data_specs()
 
# retrieve first batch from first (training) spec
next(
  iter(
    specs[0].input_fn().take(1)
    )
  )

## Define the Model

In [0]:
def get_wide_features():
    wide_columns = []

    wide_columns += [
    tf.feature_column.categorical_column_with_hash_bucket(
       key='customer_id_labeled', 
       hash_bucket_size=1000,
       dtype=tf.dtypes.int64 # create one bucket for each value from 0 to max
       )]
 
    wide_columns += [
    tf.feature_column.categorical_column_with_hash_bucket(
       key='article_id_labeled', 
       hash_bucket_size=100,
       dtype=tf.dtypes.int64 # create one bucket for each value from 0 to max
       )]
 
    # user-product cross-column (set column spec to ensure presented as int64)
    wide_columns += [
    tf.feature_column.crossed_column(
      [ tf.feature_column.categorical_column_with_identity(key='customer_id_labeled', num_buckets=np.max(cat_keys['customer_id_labeled'])+1),
        tf.feature_column.categorical_column_with_identity(key='customer_id_labeled', num_buckets=np.max(cat_keys['customer_id_labeled'])+1)
        ], 
      hash_bucket_size=1000
      )] 
 
    return wide_columns

In [0]:
def get_deep_features():
  
    deep_columns = []

    # categorical features
    for col in cat_features:

        # don't use user ID or product ID
        if col not in ['customer_id_labeled','article_id_labeled']:

            # base column definition
            col_def = tf.feature_column.categorical_column_with_identity(
                key=col, 
                num_buckets=np.max(cat_keys[col])+1 # create one bucket for each value from 0 to max
            )

            # define embedding on base column def
            deep_columns += [tf.feature_column.embedding_column(
                              col_def, 
                              dimension=int(np.max(cat_keys[col])**0.25)
                              )] 

    # continous features
    for col in num_features:
        deep_columns += [tf.feature_column.numeric_column(col)]  

    return deep_columns

In [0]:
def get_model(hidden_layers, hidden_layer_nodes_initial_count, hidden_layer_nodes_count_decline_rate, dropout_rate):  
  
    # determine hidden_units structure
    hidden_units = [None] * int(hidden_layers)
    for i in range(int(hidden_layers)):
        # decrement the nodes by the decline rate
        hidden_units[i] = int(hidden_layer_nodes_initial_count * (hidden_layer_nodes_count_decline_rate**i))

    # get features
    wide_features = get_wide_features()
    deep_features = get_deep_features()

    # define model
    estimator = tf.estimator.DNNLinearCombinedClassifier(
        linear_feature_columns=wide_features,
        #linear_optimizer=tf.keras.optimizers.Ftrl,
        linear_optimizer=tf.keras.optimizers.legacy.Ftrl,
        dnn_feature_columns=deep_features,
        dnn_hidden_units=hidden_units,
        dnn_dropout=dropout_rate,
        #dnn_optimizer=tf.keras.optimizers.Adagrad
        dnn_optimizer=tf.keras.optimizers.legacy.Adagrad
    )

    return estimator

## Tune the Model

In [0]:
# Adapted from: https://github.com/NVIDIA/DeepLearningExamples/blob/master/TensorFlow/Recommendation/WideAndDeep/utils/metrics.py
def map_custom_metric(features, labels, predictions):
  
    user_ids = tf.reshape(features['customer_id_labeled'], [-1])
    predictions = predictions['probabilities'][:, 1]

    # sort user IDs 
    sorted_ids = tf.argsort(user_ids)

    # resort values to align with sorted user IDs
    user_ids = tf.gather(user_ids, indices=sorted_ids)
    predictions = tf.gather(predictions, indices=sorted_ids)
    labels = tf.gather(labels, indices=sorted_ids)

    # get unique user IDs in dataset
    _, user_ids_idx, user_ids_items_count = tf.unique_with_counts(
        user_ids, 
        out_idx=tf.int64
    )

    # remove any user duplicates
    pad_length = 30 - tf.reduce_max(user_ids_items_count)
    pad_fn = lambda x: tf.pad(x, [(0, 0), (0, pad_length)])
    preds = tf.RaggedTensor.from_value_rowids(
        predictions, user_ids_idx).to_tensor()
    labels = tf.RaggedTensor.from_value_rowids(
        labels, user_ids_idx).to_tensor()
    labels = tf.argmax(labels, axis=1)

    # calculate average precision at k
    return {
      'map@k': tf.compat.v1.metrics.average_precision_at_k(
          predictions=pad_fn(preds),
          labels=labels,
          k=12,
          name="streaming_map")
        }

In [0]:
def train_and_evaluate_model(hparams):
  
    # retrieve the basic model
    model = get_model(
        hparams['hidden_layers'], 
        hparams['hidden_layer_nodes_initial_count'], 
        hparams['hidden_layer_nodes_count_decline_rate'], 
        hparams['dropout_rate']
    )

    # add map@k metric
    model = tf.estimator.add_metrics(model, map_custom_metric)

    # retrieve data specs
    train_spec, eval_spec = get_data_specs( int(hparams['epochs']), int(hparams['batch_size']))

    # train and evaluate
    results = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

    # return loss metric
    return results

In [0]:
hparams = {
  'hidden_layers':2,
  'hidden_layer_nodes_initial_count':100,
  'hidden_layer_nodes_count_decline_rate':0.5,
  'dropout_rate':0.25,
  'epochs':1,
  'batch_size':128
  }
 
results_train = train_and_evaluate_model(hparams)

In [0]:
results_train

In [0]:
model = get_model(
    hparams['hidden_layers'], 
    hparams['hidden_layer_nodes_initial_count'], 
    hparams['hidden_layer_nodes_count_decline_rate'], 
    hparams['dropout_rate']
    )
model = tf.estimator.add_metrics(model, map_custom_metric)
 
train_spec, eval_spec = get_data_specs(hparams['epochs'], hparams['batch_size']) 
 
results = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

# Borrowed from get_data_specs() (defined above)
# ---------------------------------------------------------
# define functions to transform data into req'ed format
def get_input_fn(dataset_context_manager):
    def _to_tuple(row): # re-structure a row as ({features}, label)
        features = {}
        for col in cat_features + num_features:
            features[col] = getattr(row, col)
        return features, getattr(row, label_col)
 
    def fn(): # called by estimator to perform row structure conversion
        return dataset_context_manager.__enter__().map(_to_tuple)
 
    return fn
# ---------------------------------------------------------
 
# define batch size and number of steps
batch_size = 128
steps = int(test_pstorm.dataset_size/batch_size)
 
# retrieve test data
test_ds = test_pstorm.make_tf_dataset(batch_size=batch_size)
 
# evaulate against test data
results = model.evaluate(get_input_fn(test_ds), steps=steps)

In [0]:
predictions = model.predict(get_input_fn(test_ds))

In [0]:
for pred in predictions:
    print(pred)

In [0]:
# results

# prediction = model.predict(specs[0].input_fn().take(1))
# print(prediction)

# get features
wide_features = get_wide_features()
deep_features = get_deep_features()
 
# use features to generate an input specification
feature_spec = tf.feature_column.make_parse_example_spec(
    wide_features + deep_features
    )
 
# make function to apply specification to incoming data
fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
    feature_spec
    )
 
# export the model
saved_model_path = model.export_saved_model(
    export_dir_base='gs://h-and-m-tx/model/model-1',
    serving_input_receiver_fn=fn
    ).decode("utf-8")

In [0]:
results